<div style="background-color: #34495e; border-bottom: 5px solid #95a5a6; padding: 10px;">
    <h2 style="color: white;">Mounting Drive</h2>
</div>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<div style="background-color: #34495e; border-bottom: 5px solid #95a5a6; padding: 10px;">
    <h2 style="color: white;">Installing the Required Libraries</h2>
</div>

In [2]:
!pip install timm grad-cam torchmetrics lime captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 30.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━

<div style="background-color: #34495e; border-bottom: 5px solid #95a5a6; padding: 10px;">
    <h2 style="color: white;">Imports</h2>
</div>

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, WeightedRandomSampler, Dataset
import torchvision.models as models
import torch.optim as optim

import os
import time
import copy
import random
import pickle

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

import cv2
import plotly.express as px
import plotly.io as pio

from PIL import Image

from collections import OrderedDict
from typing import Tuple, Union

from timm.scheduler import CosineLRScheduler
from sklearn.model_selection import StratifiedKFold
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, \
    MulticlassRecall, MulticlassF1Score, MulticlassConfusionMatrix, \
    MulticlassMatthewsCorrCoef, MulticlassSpecificity

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from captum.attr import Saliency
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
from skimage.segmentation import mark_boundaries
from sklearn.metrics import pairwise_distances

from itertools import combinations

<div style="background-color: #34495e; border-bottom: 5px solid #95a5a6; padding: 10px;">
    <h2 style="color: white;">Class Names</h2>
</div>

In [4]:
classes = {
    0: 'dyed-lifted-polyps',
    1: 'dyed-resection-margins',
    2: 'esophagitis',
    3: 'normal-cecum',
    4: 'normal-pylorus',
    5: 'normal-z-line',
    6: 'polyps',
    7: 'ulcerative-colitis',
}

<div style="background-color: #34495e; border-bottom: 5px solid #95a5a6; padding: 10px;">
    <h2 style="color: white;">Config Parameters</h2>
</div>

In [5]:
class Config:
    SEED = 2023

    # Model Parameters
    RESOLUTION = 224
    M_MLP_INPUT_DIM = 1280
    R_MLP_INPUT_DIM = 2048
    MLP_HIDDEN_DIM = 512
    MLP_DROPOUT_RATE = 0.2

    BATCH_SIZE = 64
    NUM_WORKERS = 2
    NUM_CLASSES = 8

    # Training Hyperparameters
    COOLDOWN_EPOCHS = 5
    TRAIN_EPOCHS = 25
    NUM_EPOCHS = TRAIN_EPOCHS + COOLDOWN_EPOCHS
    BASE_LR = 0.0001
    WEIGHT_DECAY = 0.00001
    PRINT_FREQ = 1024
    SAVE_FREQ = 5

    # Cosine LR Decay Parameters
    LR_MIN = 1e-6
    WARMUP_LR_INIT = 5e-5
    CYCLE_DECAY = 0.5
    CYCLE_LIMIT = 1
    WARMUP_EPOCHS = 3

    # Pretrained Models
    RESNET50_MODEL_PATH = '/content/drive/MyDrive/Kvasir_Classification/models/resnet50_model_fold_0_epoch_30.pth'
    MOBILENETV2_MODEL_PATH = '/content/drive/MyDrive/Kvasir_Classification/models/mobilenetv2_model_fold_0_epoch_30.pth'

    # Normalization
    MEAN = [0.4857, 0.3460, 0.2983]
    STD_DEV = [0.334, 0.2456, 0.2369]

    # Paths
    BASE_PATH = '/content/drive/MyDrive/Kvasir_Classification/new_data/'

    IMAGE_PATHS = ['dyed_lifted_polyps.jpg', 'dyed_resection_margins.jpg', 'esophagitis.jpg', 'normal_cecum.jpg',
                       'normal_pylorus.jpg', 'normal_z_line.jpg', 'polyps.jpg', 'ulcerative_colitis.jpg']

    RESNET50_EXPLANATION_HEATMAPS = '/content/drive/MyDrive/Kvasir_Classification/XAI Results/resnet50_heatmaps_'
    MOBILENETV2_EXPLANATION_HEATMAPS = '/content/drive/MyDrive/Kvasir_Classification/XAI Results/mobilenetv2_heatmaps_'

<div style="background-color: #34495e; border-bottom: 5px solid #95a5a6; padding: 10px;">
    <h2 style="color: white;">Defining ResNet50 Classifier</h2>
</div>

In [6]:
class ResNet50Classifier(nn.Module):

    def __init__(self, mlp_input_dim=Config.R_MLP_INPUT_DIM, mlp_hidden_dim=Config.MLP_HIDDEN_DIM, \
                 mlp_dropout_rate=Config.MLP_DROPOUT_RATE, num_classes=Config.NUM_CLASSES):

        super(ResNet50Classifier, self).__init__()

        # Load the pre-trained ResNet-50 model
        self.resnet50 = models.resnet50(pretrained=True)

        # Remove the final classification layer
        self.resnet50 = nn.Sequential(*list(self.resnet50.children())[:-1])

        # Define an MLP for classification
        self.fc = nn.Sequential(
            nn.Linear(mlp_input_dim, mlp_hidden_dim),
            nn.ReLU(),
            nn.Linear(mlp_hidden_dim, num_classes)
        )

    def forward(self, x):
        # Forward pass through ResNet-50
        resnet_features = self.resnet50(x)

        # Flatten the features
        resnet_features = resnet_features.view(resnet_features.size(0), -1)

        # Forward pass through MLP
        output = self.fc(resnet_features)
        return output

<div style="background-color: #34495e; border-bottom: 5px solid #95a5a6; padding: 10px;">
    <h2 style="color: white;">Defining MobileNetV2 Classifier</h2>
</div>

In [7]:
class MobileNetClassifier(nn.Module):
    def __init__(self, mlp_input_dim=Config.M_MLP_INPUT_DIM, mlp_hidden_dim=Config.MLP_HIDDEN_DIM, mlp_dropout_rate=Config.MLP_DROPOUT_RATE, num_classes=Config.NUM_CLASSES):
        super(MobileNetClassifier, self).__init__()

        # Load the pre-trained MobileNet V2 model
        self.mobilenet = models.mobilenet_v2(pretrained=True)

        # Remove the final classification layer
        self.mobilenet = nn.Sequential(*list(self.mobilenet.children())[:-1])

        self.reduce = nn.AdaptiveAvgPool2d(1)

        # Define an MLP for classification
        self.fc = nn.Sequential(
            nn.Linear(mlp_input_dim, mlp_hidden_dim),
            nn.ReLU(),
            nn.Linear(mlp_hidden_dim, num_classes)
        )

    def forward(self, x):
        # Forward pass through MobileNet V2
        mobilenet_features = self.mobilenet(x)

        mobilenet_features = self.reduce(mobilenet_features)

        # Flatten the features
        mobilenet_features = mobilenet_features.view(mobilenet_features.size(0), -1)

        # Forward pass through MLP
        output = self.fc(mobilenet_features)
        return output

<div style="background-color: #34495e; border-bottom: 5px solid #95a5a6; padding: 10px;">
    <h2 style="color: white;">Load PreTrained Models</h2>
</div>

In [8]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [9]:
resnet50_model = ResNet50Classifier().to(device)
checkpoint = torch.load(Config.RESNET50_MODEL_PATH, map_location=device)
resnet50_model.load_state_dict(checkpoint['model_state_dict'])
resnet50_model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 75.2MB/s]


ResNet50Classifier(
  (resnet50): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0):

In [10]:
mobilenetv2_model = MobileNetClassifier().to(device)
checkpoint = torch.load(Config.MOBILENETV2_MODEL_PATH, map_location=device)
mobilenetv2_model.load_state_dict(checkpoint['model_state_dict'])
mobilenetv2_model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 169MB/s]


MobileNetClassifier(
  (mobilenet): Sequential(
    (0): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 96, kernel_size=

<div style="background-color: #34495e; border-bottom: 5px solid #95a5a6; padding: 10px;">
    <h2 style="color: white;">Utility Functions</h2>
</div>

In [14]:
def min_max_normalize(heatmap):
    min_value = heatmap.min()
    max_value = heatmap.max()
    normalized_heatmap = (heatmap - min_value) / (max_value - min_value)
    return normalized_heatmap


def get_images(BASE_PATH, image_paths, transform):
    images = []
    for i, path in enumerate(image_paths):
        img = Image.open(BASE_PATH + path)
        img_processed = transform(img)
        images.append(img_processed)
    return images


def run_gradCAM(input_tensor, target):
    targets = [ClassifierOutputTarget(target)]
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

    input_tensor = input_tensor.squeeze(0).permute(1, 2, 0)
    min_value = torch.min(input_tensor)
    max_value = torch.max(input_tensor)
    rescaled_image = (input_tensor - min_value) / (max_value - min_value)
    rescaled_image = torch.clamp(rescaled_image, 0, 1)

    visualization = show_cam_on_image(rescaled_image.cpu().numpy(), grayscale_cam[0, :], use_rgb=True)

    return grayscale_cam[0, :], visualization


def run_saliency_map(input_tensor, target):
    input_tensor.requires_grad_(True)
    saliency_map = saliency.attribute(input_tensor, target)
    normalized_saliency_map = (saliency_map - saliency_map.min()) / (saliency_map.max() - saliency_map.min())
    visualization = normalized_saliency_map.squeeze(0).permute(1, 2, 0).cpu()
    saliency_map = saliency_map.squeeze(0).permute(1, 2, 0).cpu().numpy()
    saliency_map = np.sum(saliency_map, axis=2) / saliency_map.shape[2]
    return saliency_map, visualization


def run_integrated_gradients(inputs, model, predicted_class):
    integrated_gradients = torch.zeros_like(inputs)
    baseline = torch.zeros_like(inputs)
    m_steps = 200  # Number of steps for approximation
    alphas = torch.linspace(0.0, 1.0, m_steps)

    for alpha in alphas:
        x_interpolated = baseline + alpha * (inputs - baseline)
        x_interpolated.requires_grad_(True)

        logits = model(x_interpolated)
        predicted_class_logits = logits[0][predicted_class]

        gradient = torch.autograd.grad(predicted_class_logits, x_interpolated)[0]
        integrated_gradients += gradient / m_steps

    integrated_gradients = torch.sum(torch.abs(integrated_gradients), dim=1)
    integrated_gradients = integrated_gradients.permute(1, 2, 0).cpu().squeeze(2).numpy()

    return integrated_gradients


def get_preprocess_transform():
    transf = transforms.Compose([
        transforms.ToTensor(),
    ])
    return transf


def batch_predict_resnet50(images):
    transform = get_preprocess_transform()
    resnet50_model.eval()
    batch = torch.stack(tuple(transform(i) for i in images), dim=0)

    resnet50_model.to(device)
    batch = batch.to(device)

    logits = resnet50_model(batch)
    probs = F.softmax(logits, dim=1)
    return probs.detach().cpu().numpy()

def batch_predict_mobilenetv2(images):
    transform = get_preprocess_transform()
    mobilenetv2_model.eval()
    batch = torch.stack(tuple(transform(i) for i in images), dim=0)

    mobilenetv2_model.to(device)
    batch = batch.to(device)

    logits = mobilenetv2_model(batch)
    probs = F.softmax(logits, dim=1)
    return probs.detach().cpu().numpy()

def generate_prediction_sample(exp, exp_class, weight=0.1, show_positive=True, hide_background=True):
    image, mask = exp.get_image_and_mask(exp_class,
                                        positive_only=show_positive,
                                        num_features=3,
                                        hide_rest=hide_background,
                                        min_weight=weight)
    return image, mask


def explanation_heatmap(exp, exp_class):
    dict_heatmap = dict(exp.local_exp[exp_class])
    heatmap = np.vectorize(dict_heatmap.get)(exp.segments)
    return heatmap


def run_lime_resnet50(img):
    normalized_img = img.permute(1, 2, 0)
    exp = explainer.explain_instance(np.array(normalized_img),
                                    batch_predict_resnet50,
                                    top_labels=5,
                                    hide_color=0,
                                    num_samples=1000)

    segments = exp.segments

    for i, exp_class in enumerate(exp.top_labels):
        print(f"\n\n\033[1;31mExplanation for predicted class {exp_class}: rank {i}\033[0m")
        image, mask = generate_prediction_sample(exp, exp_class, weight=0.0001, show_positive=True, hide_background=True)
        lime_heatmap = explanation_heatmap(exp, exp_class)
        break

    return lime_heatmap, segments, image, mask


def run_lime_mobilenetv2(img):
    normalized_img = img.permute(1, 2, 0)
    exp = explainer.explain_instance(np.array(normalized_img),
                                    batch_predict_mobilenetv2,
                                    top_labels=5,
                                    hide_color=0,
                                    num_samples=1000)

    segments = exp.segments

    for i, exp_class in enumerate(exp.top_labels):
        print(f"\n\n\033[1;31mExplanation for predicted class {exp_class}: rank {i}\033[0m")
        image, mask = generate_prediction_sample(exp, exp_class, weight=0.0001, show_positive=True, hide_background=True)
        lime_heatmap = explanation_heatmap(exp, exp_class)
        break

    return lime_heatmap, segments, image, mask

<div style="background-color: #34495e; border-bottom: 5px solid #95a5a6; padding: 10px;">
    <h2 style="color: white;">ResNet50: Explanation HeatMaps</h2>
</div>

In [20]:
BASE_PATH = Config.BASE_PATH
image_paths = Config.IMAGE_PATHS

normalize = transforms.Normalize(mean=Config.MEAN,
                                 std=Config.STD_DEV)
transform = transforms.Compose([
    transforms.Resize((Config.RESOLUTION, Config.RESOLUTION)),
    transforms.ToTensor(),
    normalize,
])
images = get_images(BASE_PATH, image_paths, transform)


target_layers = [
                    resnet50_model.resnet50[7][-1]
                ]
cam = GradCAM(model=resnet50_model, target_layers=target_layers)

saliency = Saliency(resnet50_model)

explainer = lime_image.LimeImageExplainer()

default_cmap = LinearSegmentedColormap.from_list('custom blue',
                                                 [(0, '#ffffff'),
                                                  (0.25, '#000000'),
                                                  (1, '#000000')], N=256)

cosine_similarity_scores = []

for i, img in enumerate(images):
    input_tensor = img.unsqueeze(0).to(device)
    gradcam, gradcam_visualization = run_gradCAM(input_tensor, target=i)

    input_tensor = img.unsqueeze(0).to(device)
    saliency_map, saliency_map_visualization = run_saliency_map(input_tensor, target=i)

    input_tensor = img.unsqueeze(0).to(device)
    input_tensor.requires_grad_(True)
    predictions = resnet50_model(input_tensor)
    predicted_class = torch.argmax(predictions[0])
    integrated_gradients = run_integrated_gradients(input_tensor, resnet50_model, predicted_class)

    lime_heatmap, lime_segments, lime_image_map, lime_mask = run_lime_resnet50(img)

    resnet50_vis = {
        'GradCAM': gradcam_visualization,
        'Saliency Map': saliency_map_visualization,
        'Integrated Gradients': integrated_gradients,
        'LIME': lime_heatmap,
        'LIME Image': lime_image_map,
        'LIME Mask': lime_mask
    }

    fig, axes = plt.subplots(1, 5, figsize=(32, 8), dpi=600, gridspec_kw={'wspace': 0.1})

    axes[0].imshow(resnet50_vis['GradCAM'])
    axes[0].set_title('GradCAM', fontsize=22)
    axes[0].axis('off')

    axes[1].imshow(resnet50_vis['Saliency Map'])
    axes[1].set_title('Saliency Map', fontsize=22)
    axes[1].axis('off')

    axes[2].imshow(resnet50_vis['Integrated Gradients'], cmap=default_cmap, alpha=0.7)
    axes[2].set_title('Integrated Gradients', fontsize=22)
    axes[2].axis('off')

    axes[3].imshow(mark_boundaries(resnet50_vis['LIME Image'], resnet50_vis['LIME Mask']))
    axes[3].set_title('LIME: Top 3 Superpixels', fontsize=22)
    axes[3].axis('off')

    im = axes[4].imshow(resnet50_vis['LIME'], cmap='RdBu', vmin=-resnet50_vis['LIME'].max(), vmax=resnet50_vis['LIME'].max())
    axes[4].set_title('LIME', fontsize=22)
    cax = fig.add_axes([axes[4].get_position().x1 + 0.01, axes[4].get_position().y0 + 0.06, 0.01, axes[4].get_position().height])
    cbar = plt.colorbar(im, cax=cax)
    axes[4].axis('off')

    fig.suptitle(f'Model: ResNet50, Class: {classes[0]}', fontsize=24, fontweight='bold')
    plt.subplots_adjust(top=1.0)
    plt.savefig(Config.RESNET50_EXPLANATION_HEATMAPS + f'{classes[i]}.pdf', bbox_inches='tight')
    plt.show()

    # Remove this break to plot for all 8 categories
    break

  0%|          | 0/1000 [00:00<?, ?it/s]



Explanation for predicted class 0: rank 0


<div style="background-color: #34495e; border-bottom: 5px solid #95a5a6; padding: 10px;">
    <h2 style="color: white;">MobileNetV2: Explanation HeatMaps</h2>
</div>

In [21]:
BASE_PATH = Config.BASE_PATH
image_paths = Config.IMAGE_PATHS

normalize = transforms.Normalize(mean=Config.MEAN,
                                 std=Config.STD_DEV)
transform = transforms.Compose([
    transforms.Resize((Config.RESOLUTION, Config.RESOLUTION)),
    transforms.ToTensor(),
    normalize,
])
images = get_images(BASE_PATH, image_paths, transform)


target_layers = [
                    mobilenetv2_model.mobilenet[0][18][-1]
                ]
cam = GradCAM(model=mobilenetv2_model, target_layers=target_layers)

saliency = Saliency(mobilenetv2_model)

explainer = lime_image.LimeImageExplainer()

default_cmap = LinearSegmentedColormap.from_list('custom blue',
                                                 [(0, '#ffffff'),
                                                  (0.25, '#000000'),
                                                  (1, '#000000')], N=256)

cosine_similarity_scores = []

for i, img in enumerate(images):
    input_tensor = img.unsqueeze(0).to(device)
    gradcam, gradcam_visualization = run_gradCAM(input_tensor, target=i)

    input_tensor = img.unsqueeze(0).to(device)
    saliency_map, saliency_map_visualization = run_saliency_map(input_tensor, target=i)

    input_tensor = img.unsqueeze(0).to(device)
    input_tensor.requires_grad_(True)
    predictions = mobilenetv2_model(input_tensor)
    predicted_class = torch.argmax(predictions[0])
    integrated_gradients = run_integrated_gradients(input_tensor, mobilenetv2_model, predicted_class)

    lime_heatmap, lime_segments, lime_image_map, lime_mask = run_lime_mobilenetv2(img)

    mobilenetv2_vis = {
        'GradCAM': gradcam_visualization,
        'Saliency Map': saliency_map_visualization,
        'Integrated Gradients': integrated_gradients,
        'LIME': lime_heatmap,
        'LIME Image': lime_image_map,
        'LIME Mask': lime_mask
    }

    fig, axes = plt.subplots(1, 5, figsize=(32, 8), dpi=600, gridspec_kw={'wspace': 0.1})

    axes[0].imshow(mobilenetv2_vis['GradCAM'])
    axes[0].set_title('GradCAM', fontsize=22)
    axes[0].axis('off')

    axes[1].imshow(mobilenetv2_vis['Saliency Map'])
    axes[1].set_title('Saliency Map', fontsize=22)
    axes[1].axis('off')

    axes[2].imshow(mobilenetv2_vis['Integrated Gradients'], cmap=default_cmap, alpha=0.7)
    axes[2].set_title('Integrated Gradients', fontsize=22)
    axes[2].axis('off')

    axes[3].imshow(mark_boundaries(mobilenetv2_vis['LIME Image'], mobilenetv2_vis['LIME Mask']))
    axes[3].set_title('LIME: Top 3 Superpixels', fontsize=22)
    axes[3].axis('off')

    im = axes[4].imshow(mobilenetv2_vis['LIME'], cmap='RdBu', vmin=-mobilenetv2_vis['LIME'].max(), vmax=mobilenetv2_vis['LIME'].max())
    axes[4].set_title('LIME', fontsize=22)
    cax = fig.add_axes([axes[4].get_position().x1 + 0.01, axes[4].get_position().y0 + 0.06, 0.01, axes[4].get_position().height])
    cbar = plt.colorbar(im, cax=cax)
    axes[4].axis('off')

    fig.suptitle(f'Model: MobileNetV2, Class: {classes[0]}', fontsize=24, fontweight='bold')
    plt.subplots_adjust(top=1.0)
    plt.savefig(Config.MOBILENETV2_EXPLANATION_HEATMAPS + f'{classes[i]}.pdf', bbox_inches='tight')
    plt.show()

    # Remove this break to plot for all 8 categories
    break

  0%|          | 0/1000 [00:00<?, ?it/s]



Explanation for predicted class 0: rank 0
